### Data manipulation example

We read in a SPLICE dataset and format it ready for inclusion in our NN model.
Where there is no transaction in a development period, that development period with have no record. Settled claims are also not included.

Steps:  

1. Read in the data
2. Create additional variables
3. Apply cutoff
4. Make sure there is one record for every development period
5. Create variables and define which ones you want to use in the model
6. Create train and test datasets

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.nn import functional as F

In [19]:
dirname="/home/nigel/git/MLR_working_party/01_data/"
filename="data_origframework_nofills_nosttl.csv"

pd.options.display.float_format = '{:,.2f}'.format

### 1. Read in the data from SPLICE  
Here we just use the paid data. Case estimate data is also available in the incurred_1.csv file but we do not use it here

In [6]:
transactions = pd.read_csv(
    f"https://raw.githubusercontent.com/agi-lab/SPLICE/main/datasets/complexity_1/payment_1.csv"
)
transactions

,Unnamed: 0,claim_no,pmt_no,occurrence_period,occurrence_time,claim_size,notidel,setldel,payment_time,payment_period,payment_size,payment_inflated,payment_delay
0,1,1,1,1,0.73,"232,310.09",0.66,23.21,5.33,6,"13,226.34","13,226.34",3.93
1,2,1,2,1,0.73,"232,310.09",0.66,23.21,10.09,11,"15,685.86","15,685.86",4.76
2,3,1,3,1,0.73,"232,310.09",0.66,23.21,18.02,19,"14,643.28","14,643.28",7.93
3,4,1,4,1,0.73,"232,310.09",0.66,23.21,22.82,23,"170,041.89","170,041.89",4.79
4,5,1,5,1,0.73,"232,310.09",0.66,23.21,24.61,25,"18,712.71","18,712.71",1.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19317,19318,3663,2,40,39.87,"40,653.72",2.29,10.67,46.40,47,"2,213.88","2,213.88",1.84
19318,19319,3663,3,40,39.87,"40,653.72",2.29,10.67,48.45,49,"2,292.75","2,292.75",2.06
19319,19320,3663,4,40,39.87,"40,653.72",2.29,10.67,50.24,51,"2,367.32","2,367.32",1.78
19320,19321,3663,5,40,39.87,"40,653.72",2.29,10.67,51.76,52,"28,576.07","28,576.07",1.52


### 2. Create additional variables

In [7]:
transactions["noti_period"] = np.ceil(transactions["occurrence_time"] + transactions["notidel"]).astype('int')
transactions["settle_period"] = np.ceil(transactions["occurrence_time"] + transactions["notidel"] + transactions["setldel"]).astype('int')

### 3. Apply cutoff

In [8]:
# Apply a maximum development period
maxdev=40

transactions["development_period"] = np.minimum(transactions["payment_period"] - transactions["occurrence_period"], maxdev)  

transactions

,Unnamed: 0,claim_no,pmt_no,occurrence_period,occurrence_time,claim_size,notidel,setldel,payment_time,payment_period,payment_size,payment_inflated,payment_delay,noti_period,settle_period,development_period
0,1,1,1,1,0.73,"232,310.09",0.66,23.21,5.33,6,"13,226.34","13,226.34",3.93,2,25,5
1,2,1,2,1,0.73,"232,310.09",0.66,23.21,10.09,11,"15,685.86","15,685.86",4.76,2,25,10
2,3,1,3,1,0.73,"232,310.09",0.66,23.21,18.02,19,"14,643.28","14,643.28",7.93,2,25,18
3,4,1,4,1,0.73,"232,310.09",0.66,23.21,22.82,23,"170,041.89","170,041.89",4.79,2,25,22
4,5,1,5,1,0.73,"232,310.09",0.66,23.21,24.61,25,"18,712.71","18,712.71",1.79,2,25,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19317,19318,3663,2,40,39.87,"40,653.72",2.29,10.67,46.40,47,"2,213.88","2,213.88",1.84,43,53,7
19318,19319,3663,3,40,39.87,"40,653.72",2.29,10.67,48.45,49,"2,292.75","2,292.75",2.06,43,53,9
19319,19320,3663,4,40,39.87,"40,653.72",2.29,10.67,50.24,51,"2,367.32","2,367.32",1.78,43,53,11
19320,19321,3663,5,40,39.87,"40,653.72",2.29,10.67,51.76,52,"28,576.07","28,576.07",1.52,43,53,12


Only include records for a claim that is after the notification period

In [42]:
#dat = dat.loc[dat.payment_period >= dat.noti_period].copy()
#dat

### 5. Create variables and define which ones you want to use in the model

In [9]:
dat=transactions

# Clean close to zero values
dat["payment_size"] = np.where(abs(dat.payment_size) < 1e-2, 0.0, dat.payment_size)

#    payment_period=lambda df: (df.occurrence_period + df.development_period),
dat["is_settled"]=((dat.occurrence_period + dat.development_period) >= dat.settle_period)

# Cumulative payments
dat["payment_size_cumulative"] = dat[["claim_no", "payment_size"]].groupby('claim_no').cumsum()
dat["log1_paid_cumulative"] = np.log1p(dat.payment_size_cumulative)

dat["pmt_no"] = dat.groupby("claim_no")["pmt_no"].cummax()

Show records just for an individual claim - here claim_no 1

In [10]:
dat[dat['claim_no'] == 1]

,Unnamed: 0,claim_no,pmt_no,occurrence_period,occurrence_time,claim_size,notidel,setldel,payment_time,payment_period,payment_size,payment_inflated,payment_delay,noti_period,settle_period,development_period,is_settled,payment_size_cumulative,log1_paid_cumulative
0,1,1,1,1,0.73,"232,310.09",0.66,23.21,5.33,6,"13,226.34","13,226.34",3.93,2,25,5,False,"13,226.34",9.49
1,2,1,2,1,0.73,"232,310.09",0.66,23.21,10.09,11,"15,685.86","15,685.86",4.76,2,25,10,False,"28,912.20",10.27
2,3,1,3,1,0.73,"232,310.09",0.66,23.21,18.02,19,"14,643.28","14,643.28",7.93,2,25,18,False,"43,555.48",10.68
3,4,1,4,1,0.73,"232,310.09",0.66,23.21,22.82,23,"170,041.89","170,041.89",4.79,2,25,22,False,"213,597.38",12.27
4,5,1,5,1,0.73,"232,310.09",0.66,23.21,24.61,25,"18,712.71","18,712.71",1.79,2,25,24,True,"232,310.09",12.36


Define which variables you want to use in the model

output_field and youtput are also created to define the y variate for use in the model

### Only include records for where the claim is not settled

In [11]:
dat = dat.loc[dat.is_settled == False]
dat[dat['claim_no'] == 1]

,Unnamed: 0,claim_no,pmt_no,occurrence_period,occurrence_time,claim_size,notidel,setldel,payment_time,payment_period,payment_size,payment_inflated,payment_delay,noti_period,settle_period,development_period,is_settled,payment_size_cumulative,log1_paid_cumulative
0,1,1,1,1,0.73,"232,310.09",0.66,23.21,5.33,6,"13,226.34","13,226.34",3.93,2,25,5,False,"13,226.34",9.49
1,2,1,2,1,0.73,"232,310.09",0.66,23.21,10.09,11,"15,685.86","15,685.86",4.76,2,25,10,False,"28,912.20",10.27
2,3,1,3,1,0.73,"232,310.09",0.66,23.21,18.02,19,"14,643.28","14,643.28",7.93,2,25,18,False,"43,555.48",10.68
3,4,1,4,1,0.73,"232,310.09",0.66,23.21,22.82,23,"170,041.89","170,041.89",4.79,2,25,22,False,"213,597.38",12.27


In [12]:
# Potential features for model later:
data_cols = [
    "claim_no",
    "occurrence_time", 
    "notidel", 
    "development_period", 
    "pmt_no",
    "log1_paid_cumulative",
]

list_of_features = data_cols
output_field = ["claim_size"]
youtput="claim_size"

dat.loc[:, data_cols + [youtput]]
#dat.loc[:, list_of_features + output_field]

,claim_no,occurrence_time,notidel,development_period,pmt_no,log1_paid_cumulative,claim_size
0,1,0.73,0.66,5,1,9.49,"232,310.09"
1,1,0.73,0.66,10,2,10.27,"232,310.09"
2,1,0.73,0.66,18,3,10.68,"232,310.09"
3,1,0.73,0.66,22,4,12.27,"232,310.09"
5,2,0.33,0.74,3,1,9.55,"237,675.12"
...,...,...,...,...,...,...,...
19316,3663,39.87,2.29,5,1,7.87,"40,653.72"
19317,3663,39.87,2.29,7,2,8.48,"40,653.72"
19318,3663,39.87,2.29,9,3,8.87,"40,653.72"
19319,3663,39.87,2.29,11,4,9.16,"40,653.72"


## 6. Create train and test datasets.  
We provide three types of indicators here for reference, but in our NN example we only use train_ind  

* train_ind creates a split by claim - 'rectangular data'
* train_ind_time is what you would use if you wanted to split the datasets by calendar period
* cv_ind is what you might use if you were doing cross validation

In [13]:
cutoff=maxdev

dat["train_ind"] = (dat.claim_no % 10 >= 4)
dat["train_ind_time"] = (dat.payment_period <= cutoff)
dat["cv_ind"] = dat.payment_period % 5
dat

<positron-console-cell-13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-13>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-13>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,claim_no,pmt_no,occurrence_period,occurrence_time,claim_size,notidel,setldel,payment_time,payment_period,...,payment_delay,noti_period,settle_period,development_period,is_settled,payment_size_cumulative,log1_paid_cumulative,train_ind,train_ind_time,cv_ind
0,1,1,1,1,0.73,"232,310.09",0.66,23.21,5.33,6,...,3.93,2,25,5,False,"13,226.34",9.49,False,True,1
1,2,1,2,1,0.73,"232,310.09",0.66,23.21,10.09,11,...,4.76,2,25,10,False,"28,912.20",10.27,False,True,1
2,3,1,3,1,0.73,"232,310.09",0.66,23.21,18.02,19,...,7.93,2,25,18,False,"43,555.48",10.68,False,True,4
3,4,1,4,1,0.73,"232,310.09",0.66,23.21,22.82,23,...,4.79,2,25,22,False,"213,597.38",12.27,False,True,3
5,6,2,1,1,0.33,"237,675.12",0.74,22.15,3.46,4,...,2.38,2,24,3,False,"14,014.05",9.55,False,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19316,19317,3663,1,40,39.87,"40,653.72",2.29,10.67,44.56,45,...,2.40,43,53,5,False,"2,618.23",7.87,False,False,0
19317,19318,3663,2,40,39.87,"40,653.72",2.29,10.67,46.40,47,...,1.84,43,53,7,False,"4,832.11",8.48,False,False,2
19318,19319,3663,3,40,39.87,"40,653.72",2.29,10.67,48.45,49,...,2.06,43,53,9,False,"7,124.86",8.87,False,False,4
19319,19320,3663,4,40,39.87,"40,653.72",2.29,10.67,50.24,51,...,1.78,43,53,11,False,"9,492.18",9.16,False,False,1


In [14]:
X_train = (dat.loc[(dat.train_ind == 1), list_of_features])
y_train = (dat.loc[(dat.train_ind == 1), youtput])

X_test = (dat.loc[(dat.train_ind == 0), list_of_features])
y_test = (dat.loc[(dat.train_ind == 0), youtput])

X = (dat.loc[:, list_of_features])
y = (dat.loc[:, youtput])

Save file to disk

In [20]:
dat.to_csv(
    dirname + filename
)

In [18]:
filename

'data_origframework_nofills_nosttl.csv'